In [69]:
import Control.Monad
import Test.QuickCheck
import Test.QuickCheck.Checkers
import Test.QuickCheck.Classes

In [3]:
bind :: Monad m => m a -> (a -> m b) -> m b
bind a f = join (fmap f a)

<h4>Simple comparison between Functors and Monads</h4>

In [22]:
twiceWhenEven = fmap (\x -> if x `mod` 2 == 1 then [] else [x*x, x*x])

In [23]:
twiceWhenEven [1,3]

[[],[]]

In [17]:
twiceWhenEvenMonadic :: [Integer] -> [Integer]
twiceWhenEvenMonadic xs = do
    x <- xs
    if even x
        then [x*x, x*x]
        else [x*x]

In [18]:
twiceWhenEvenMonadic [1..3]

[1,4,4,9]

In [19]:
twiceWhenEvenMonadic' :: [Integer] -> [Integer]
twiceWhenEvenMonadic' xs = do
    x <- xs
    if even x
        then [x*x, x*x]
        else []

In [21]:
twiceWhenEvenMonadic' [1,3]

[]

<h4>Cow example (compare with example in Applicatives)</h4>

In [24]:
data Cow = Cow { name:: String, age:: Int, weight:: Int } deriving (Eq, Show)

In [25]:
noEmpty :: String -> Maybe String
noEmpty "" = Nothing
noEmpty str = Just str

In [26]:
noNegative :: Int -> Maybe Int
noNegative n 
    | n >= 0     = Just n
    | otherwise  = Nothing

In [27]:
weightCheck :: Cow -> Maybe Cow
weightCheck c = 
    let w = weight c
        n = name c
    in if n == "Bess" && w > 499
        then Nothing
        else Just c

In [28]:
mkSphericalCow :: String -> Int -> Int -> Maybe Cow
mkSphericalCow name' age' wt' = do
    maybeName <- noEmpty name'
    maybeAge <- noNegative age'
    maybeWt <- noNegative wt'
    weightCheck (Cow maybeName maybeAge maybeWt)

<h4>Another example</h4>

In [31]:
f :: Integer -> Maybe Integer
f 0 = Nothing
f n = Just n

In [32]:
g :: Integer -> Maybe Integer
g i = if even i then Just (i+1) else Nothing

In [33]:
h :: Integer -> Maybe String
h i = Just (show i)

In [34]:
doSomething' n = do
    a <- f n
    b <- g a
    c <- h b
    pure (a, b, c)

In [35]:
doSomething' 100

Just (100,101,"101")

<h4>Either Monad example</h4>

In [37]:
type Founded = Int
type Coders = Int

In [45]:
data SoftwareShop = Shop { founded :: Founded, programmers :: Coders } deriving (Eq, Show)

In [48]:
data FoundedError =
    NegativeYears Founded
    | TooManyYears Founded
    | NegativeCoders Coders
    | TooManyCoders Coders
    | TooManyCodersForYears Founded Coders
    deriving (Eq, Show)

In [49]:
validateFounded :: Int -> Either FoundedError Founded
validateFounded n
    | n < 0 = Left $ NegativeYears n
    | n > 500 = Left $ TooManyYears n
    | otherwise = Right n

In [50]:
validateCoders :: Int -> Either FoundedError Coders
validateCoders n
    | n < 0 = Left $ NegativeCoders n
    | n > 5000 = Left $ TooManyCoders n
    | otherwise = Right n

In [51]:
mkSoftware :: Int -> Int -> Either FoundedError SoftwareShop
mkSoftware yrs coders = do
    founded <- validateFounded yrs
    programmers <- validateCoders coders
    if programmers > div founded 10
        then Left $
                TooManyCodersForYears founded programmers
        else Right $ Shop founded programmers

In [57]:
tooManyCodersCheck :: Founded -> Coders -> Either FoundedError SoftwareShop
tooManyCodersCheck yrs coders = if coders > div yrs 10 
                                    then Left $ TooManyCodersForYears yrs coders 
                                    else Right $ Shop yrs coders

mkSoftware' :: Int -> Int -> Either FoundedError SoftwareShop
mkSoftware' yrs coders = validateFounded yrs 
                            >>= \years -> validateCoders coders
                                >>= \cdrs -> tooManyCodersCheck years cdrs

In [63]:
mkSoftware 0 500

Left (TooManyCodersForYears 0 500)

<h4>Sum (Either Synonym)</h4>

In [75]:
data Sum a b = First a | Second b deriving (Eq, Show)

instance (Arbitrary a, Arbitrary b) => Arbitrary (Sum a b) where
    arbitrary = frequency[(1, fmap First arbitrary), (3, fmap Second arbitrary)]
    
instance (Eq a, Eq b) => EqProp (Sum a b) where
    (=-=) = eq

In [76]:
instance Functor (Sum a) where
    fmap _ (First a) = First a
    fmap f (Second b) = Second (f b)

In [78]:
trigger :: Sum String (String, Int, String)
trigger = undefined

verboseBatch (functor trigger)


functor:
  identity: +++ OK, passed 500 tests.
  compose:  +++ OK, passed 500 tests.

In [81]:
instance Applicative (Sum a) where
    pure = Second
    (<*>) (First f) _ = First f
    (<*>) _ (First x) = First x
    (<*>) (Second f) x = fmap f x

In [83]:
verboseBatch (applicative trigger)


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

In [90]:
instance Monad (Sum a) where
    return = pure
    (>>=) (First a) _ = First a
    (>>=) (Second b) f = f b

In [91]:
verboseBatch (monad trigger)


monad laws:
  left  identity: +++ OK, passed 500 tests.
  right identity: +++ OK, passed 500 tests.
  associativity:  +++ OK, passed 500 tests.

<h4>Bad Monad</h4>

In [96]:
data CountMe a = CountMe Integer a deriving (Eq, Show)

In [106]:
instance Functor CountMe where
    fmap f (CountMe i a) = CountMe i (f a)

In [107]:
instance Applicative CountMe where
    pure = CountMe 0
    (<*>) (CountMe n f) (CountMe n' a) = CountMe (n + n') (f a)

In [114]:
instance Monad CountMe where
    return = pure
    (>>=) (CountMe n a) f = CountMe (n + n') b where
                            CountMe n' b = f a

In [115]:
instance Arbitrary a => Arbitrary (CountMe a) where
    arbitrary = CountMe <$> arbitrary <*> arbitrary
    
instance Eq a => EqProp (CountMe a) where
    (=-=) = eq

In [116]:
trigger :: CountMe (Int, String, Int)
trigger = undefined

quickBatch $ functor trigger
quickBatch $ applicative trigger
quickBatch $ monad trigger


functor:
  identity: +++ OK, passed 500 tests.
  compose:  +++ OK, passed 500 tests.


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.


monad laws:
  left  identity: +++ OK, passed 500 tests.
  right identity: +++ OK, passed 500 tests.
  associativity:  +++ OK, passed 500 tests.

<h4>Monadic composition</h4>

In [117]:
sayHi :: String -> IO String
sayHi greeting = do
    putStrLn greeting
    getLine
    
readM :: Read a => String -> IO a
readM = return . read

getAge :: String -> IO Int
getAge = sayHi >=> readM

askForAge :: IO Int
askForAge = getAge "Hello! How old are you? "

<h3>Exercises</h3>

<h4>Nope</h4>

In [119]:
data Nope a = NopeDotJpg deriving (Eq, Show)

instance Arbitrary (Nope a) where
    arbitrary = return NopeDotJpg
    
instance EqProp (Nope a) where
    (=-=) = eq

In [120]:
instance Functor Nope where
    fmap _ NopeDotJpg = NopeDotJpg

In [121]:
instance Applicative Nope where
    pure _ = NopeDotJpg
    (<*>) _ _ = NopeDotJpg

In [124]:
instance Monad Nope where
    return = pure
    (>>=) _ _ = NopeDotJpg

In [125]:
trigger :: Nope (Int, String, Int)
trigger = undefined

quickBatch $ functor trigger
quickBatch $ applicative trigger
quickBatch $ monad trigger


functor:
  identity: +++ OK, passed 500 tests.
  compose:  +++ OK, passed 500 tests.


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.


monad laws:
  left  identity: +++ OK, passed 500 tests.
  right identity: +++ OK, passed 500 tests.
  associativity:  +++ OK, passed 500 tests.

<h4>Identity</h4>

In [127]:
newtype Identity a = Identity a deriving (Eq, Show, Ord)

In [128]:
instance Arbitrary (Nope a) where
    arbitrary = return NopeDotJpg
    
instance EqProp (Nope a) where
    (=-=) = eq

In [129]:
instance Functor Identity where
    fmap f (Identity a) = Identity (f a)

In [130]:
instance Applicative Identity where
    pure = Identity
    (<*>) (Identity f) (Identity a) = Identity (f a)

In [132]:
instance Monad Identity where
    return = pure
    (>>=) (Identity a) f = f a

In [133]:
trigger :: Nope (Int, String, Int)
trigger = undefined

quickBatch $ functor trigger
quickBatch $ applicative trigger
quickBatch $ monad trigger


functor:
  identity: +++ OK, passed 500 tests.
  compose:  +++ OK, passed 500 tests.


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.


monad laws:
  left  identity: +++ OK, passed 500 tests.
  right identity: +++ OK, passed 500 tests.
  associativity:  +++ OK, passed 500 tests.

<h4>Listy (Synonym of List)</h4>

In [134]:
data Listy a = Nil | Cons {head':: a, tail':: Listy a} deriving (Eq, Show)

instance Arbitrary a => Arbitrary (Listy a) where
    arbitrary = frequency [(1, pure Nil), (5, Cons <$> arbitrary <*> arbitrary)]
    
instance Eq a => EqProp (Listy a) where
    (=-=) = eq

In [135]:
instance Semigroup (Listy a) where
    (<>) Nil ys = ys
    (<>) (Cons x xs) ys = Cons x (xs <> ys)
    
instance Monoid (Listy a) where
    mempty = Nil
    mappend = (<>)

In [136]:
instance Functor Listy where
    fmap _ Nil = Nil
    fmap f (Cons x xs) = Cons (f x) (fmap f xs)

In [137]:
instance Applicative Listy where
    pure x = Cons x Nil
    (<*>) _ Nil = Nil
    (<*>) Nil _ = Nil
    (<*>) (Cons f fs) x = (f <$> x) <> (fs <*> x)

In [144]:
instance Monad Listy where
    return = pure
    (>>=) Nil _ = Nil
    (>>=) (Cons x xs) f = f x <> (xs >>= f)

In [145]:
trigger :: Listy (String, String, String)
trigger = undefined

quickBatch $ monad trigger


monad laws:
  left  identity: +++ OK, passed 500 tests.
  right identity: +++ OK, passed 500 tests.
  associativity:  +++ OK, passed 500 tests.

In [140]:
:t (>>=)

(>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b